In [9]:
from pyspark.sql import SparkSession

appName = "PySpark Example"
master = "local"
# Create Spark session
spark = SparkSession.builder \
    .appName(appName) \
    .master(master) \
    .config("spark.jars", "/usr/local/spark/jars/mysql-connector-java-8.0.30.jar") \
    .getOrCreate()



In [13]:

server = "localhost"
port = 3306
database = "nation"
jdbc_url = f"jdbc:mysql://{server}:{port}/{database}?permitMysqlScheme"


user = "alinizar"
password = "alinizarpwd"
jdbc_driver = "com.mysql.cj.jdbc.Driver"


properties = {
    "user": user,
    "password": password,
    "driver": jdbc_driver
}


In [14]:

continents_df = spark.read.jdbc(jdbc_url, "(select * from continents) tab", properties=properties)
countries_df = spark.read.jdbc(jdbc_url, "(select * from countries) tab", properties=properties)
country_languages_df = spark.read.jdbc(jdbc_url, "(select * from country_languages) tab", properties=properties)
country_stats_df = spark.read.jdbc(jdbc_url, "(select * from country_stats) tab", properties=properties)
languages_df = spark.read.jdbc(jdbc_url, "(select * from languages) tab", properties=properties)
regions_df = spark.read.jdbc(jdbc_url, "(select * from regions) tab", properties=properties)
vips_df = spark.read.jdbc(jdbc_url, "(select * from vips) tab", properties=properties)
guests_df = spark.read.jdbc(jdbc_url, "(select * from guests) tab", properties=properties)



In [15]:
joined_df = (
    countries_df
    .join(regions_df, countries_df.region_id == regions_df.region_id, "left")
    .join(continents_df, regions_df.continent_id == continents_df.continent_id, "left")
    .join(country_languages_df, countries_df.country_id == country_languages_df.country_id, "left")
    .join(languages_df, country_languages_df.language_id == languages_df.language_id, "left")
    .join(country_stats_df, countries_df.country_id == country_stats_df.country_id, "left")
)

# Show the joined DataFrame
joined_df.show()

+----------+-----+---------+------------+-------------+-------------+---------+---------+--------------+------------+------------+------+----------+-----------+--------+-----------+--------+----------+----+----------+-----------+
|country_id| name|     area|national_day|country_code2|country_code3|region_id|region_id|          name|continent_id|continent_id|  name|country_id|language_id|official|language_id|language|country_id|year|population|        gdp|
+----------+-----+---------+------------+-------------+-------------+---------+---------+--------------+------------+------------+------+----------+-----------+--------+-----------+--------+----------+----+----------+-----------+
|        20|Benin|112622.00|  1960-08-01|           BJ|          BEN|       12|       12|Western Africa|           3|           3|Africa|        20|         54|   false|         54|   Somba|        20|2018|  11485048|10358985583|
|        20|Benin|112622.00|  1960-08-01|           BJ|          BEN|       12| 